In [5]:
import pandas as pd
import pymysql
import pandas.io.sql as sql

### 数据获取

In [16]:
# 从mysql 读取已经获取的数据到  df 中
try:
    conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='111111', db='STOCK', charset='utf8')
    df = sql.read_sql('select * from Dfcf_allfund', conn) #将数据库读入dataframe
    conn.close()
except pymysql.Error as e:
    print('Mysql Error %d: %s' %(e.args[0], e.args[1]))

### 数据清理

In [14]:
# 设置name 为index
df.drop(['id'],axis=1,inplace=True)
data = df.set_index(['name']).copy()
data.head(3)

,1m,3m,6m,1y,3y,fs
name,,,,,,
易方达天天理财货币A,0.35%,3.41%,1.04%,11.15%,1.97%,18.51%
易方达天天理财货币R,0.37%,3.67%,1.11%,11.98%,2.10%,19.83%
华夏大盘精选,-1.03%,16.19%,7.99%,64.20%,16.11%,1955.34%


In [20]:
def clean_num_in_column(column):
    return column.apply(drop_percent_sign)

# state中“%”替换“空”，并设为float，以便后续计算
def drop_percent_sign(state): 
    if state.endswith('%'):
        return float(state.replace('%','')) 

In [21]:
data_drop_percent = data.apply(clean_num_in_column)

### 数据清理完毕

In [23]:
data_drop_percent.head(1)

,1m,3m,6m,1y,3y,fs
name,,,,,,
易方达天天理财货币A,0.35,3.41,1.04,11.15,1.97,18.51


In [74]:
# 取长期收益前100，中期（3个月）收益前200，短期（1个月）收益前100名，单独存到不同 DataFrame中，为后续统计不同时间段最优基金
top1 = 100
top2 = 200
top3 = 300
m1 = data_drop_percent.sort_values(by=['1m'],ascending=False).head(top3)
m3 = data_drop_percent.sort_values(by=['3m'],ascending=False).head(top2)
m6 = data_drop_percent.sort_values(by=['6m'],ascending=False).head(top1)
y1 = data_drop_percent.sort_values(by=['1y'],ascending=False).head(top1)
y3 = data_drop_percent.sort_values(by=['3y'],ascending=False).head(top1)
fs = data_drop_percent.sort_values(by=['fs'],ascending=False).head(top1)

In [63]:
# 所有基金各周期排名
ranks = data_drop_percent.rank(ascending=False)

In [75]:
ranks.head()
#ranks.sort_values(['6m']).iloc[0:100,:]

,1m,3m,6m,1y,3y,fs
name,,,,,,
易方达天天理财货币A,3348.0,1914.0,4293.5,1756.5,3049.5,2084.5
易方达天天理财货币R,3129.0,1735.5,4015.0,1681.5,2793.0,1995.5
华夏大盘精选,5616.5,509.5,833.0,431.5,324.0,1.0
华夏成长,5548.5,1588.5,2053.5,1250.5,977.0,51.0
华夏聚利债券,2032.0,3545.5,2609.0,1555.0,3897.0,2183.0


In [76]:
# 按照各周期组合，组成不同周期前100名的新DF，以便筛选最优基金
total_keys = ['1m','3m','6m','1y','3y','fs']
near2 = pd.merge(m1, m3, left_index=True, how='inner', on=total_keys)
near3 = pd.merge(m6, near2, left_index=True, how='inner', on=total_keys)
near4 = pd.merge(y1, near3, left_index=True, how='inner', on=total_keys)
near5 = pd.merge(y3, near4, left_index=True, how='inner', on=total_keys)
total = pd.merge(fs, near5, left_index=True, how='inner', on=total_keys)

In [77]:
total
# 所有周期排名均为前100的基金：无

,1m,3m,6m,1y,3y,fs
name,,,,,,


In [78]:
near5
# 近3年每次排名均为前100的基金：无

,1m,3m,6m,1y,3y,fs
name,,,,,,
南方消费进取,7.27,63.15,30.83,241.94,37.43,231.36


In [79]:
near4.sort_values('1y',ascending=False)
# 近1年每次排名均为前100的基金：无

,1m,3m,6m,1y,3y,fs
name,,,,,,
南方消费进取,7.27,63.15,30.83,241.94,37.43,231.36
浙商进取,5.16,28.26,23.02,119.68,22.22,50.70


In [80]:
near3.sort_values('6m',ascending=False).head()
# 近6个月，3个月，1个月名均为前100的基金

,1m,3m,6m,1y,3y,fs
name,,,,,,
中融中证煤炭指数分级B,28.28,63.70,102.97,NaN,51.20,-24.70
中融国证钢铁行业指数分级B,52.66,96.57,69.18,NaN,46.59,-82.40
信诚中证800有色指数分级B,28.50,54.89,65.03,-24.36,50.42,-32.22
招商中证煤炭等权指数分级B,25.88,49.03,61.31,NaN,32.97,-48.16
鹏华钢铁分级B,42.28,87.89,57.82,NaN,43.57,27.20


In [81]:
near2.sort_values('3m',ascending=False).head()
# 近3个月，1个月名均为前100的基金

,1m,3m,6m,1y,3y,fs
name,,,,,,
中融国证钢铁行业指数分级B,52.66,96.57,69.18,NaN,46.59,-82.4
招商中证白酒指数分级B,21.54,95.72,40.28,NaN,82.12,78.3
鹏华钢铁分级B,42.28,87.89,57.82,NaN,43.57,27.2
鹏华酒分级B,18.22,73.18,31.28,NaN,54.48,25.9
方正富邦保险主题指数分级B,7.38,69.39,46.05,NaN,39.78,36.7


### 已知基金名称，比较各周期绝对收益

In [82]:
data_drop_percent.loc[['南方消费进取','博时主题行业混合(LOF)','汇丰晋信大盘股票A','广发行业领先混合A','富国低碳环保混合','易方达消费行业','景顺长城新兴成长混合'],:]

,1m,3m,6m,1y,3y,fs
name,,,,,,
南方消费进取,7.27,63.15,30.83,241.94,37.43,231.36
博时主题行业混合(LOF),-0.23,16.23,6.51,102.17,11.06,1079.15
汇丰晋信大盘股票A,1.11,32.32,11.98,196.35,14.90,225.08
广发行业领先混合A,-0.78,8.46,7.86,87.87,14.54,104.78
富国低碳环保混合,-4.70,17.27,1.78,130.04,19.62,151.20
易方达消费行业,1.72,37.25,11.90,100.21,29.72,89.00
景顺长城新兴成长混合,1.32,21.85,9.32,70.19,24.76,161.59
